In this program we will solve a reaction diffusion equation based on your imput. 
The equation we work with is as follows:

# Diffusion equation: 
\begin{equation}
    u_{i,k+1} - D(\Delta t)\frac{u_{i-1,k+1}-2u_{i,k+1}+u_{i+1,k+1}}{2(\Delta x)^2}
    = u_{i,k} + D(\Delta t)\frac{u_{i-1,k}-2u_{i,k}+u_{i+1,k}}{2(\Delta x)^2} + (\Delta t)f(x_i,t_k,u_{i,k})
\end{equation}

In [2]:
using Plots

In [1]:
abstract type BoundaryCondition
end

type Dirichlet <: BoundaryCondition 
    ū₀::Float64
    ūₗ::Float64
end

type Neumann <: BoundaryCondition
    ū₀::Float64
    ūₗ::Float64
end 

type Periodic <: BoundaryCondition
    ū₀::Float64   # ū₀ = ūₗ
    ūₗ::Float64
end 

In [3]:
type Discretization
    Nₜ::Float64 
    Nₓ::Float64 
    Δx::Float64
    Δt::Float64
end 

type SpaceTime
    L::Float64
    tf::Float64
end

In [4]:
function build_tri_diagonal_matrix(discretization::Discretization, λ::Float64, bc::BoundaryCondition)
    # number of unknowns
    matrix_dim = discretization.Nₓ
    
    if typeof(bc) == Dirichlet
        matrix_dim -= 2
    end
    
    if typeof(bc) == Periodic
        matrix_dim -= 1
    end
        
    A = zeros(matrix_dim, matrix_dim)
    
    for i = 1:matrix_dim
        A[i, i] = 1 + 2 * λ
    end
        
    for i = 1:matrix_dim - 1
        A[i  , i+1] = -λ
        A[i+1, i  ] = -λ
    end
    
    if typeof(bc) == Periodic
        A[1, end] = -λ
        A[end, 1] = -λ
    end
        
    if typeof(bc) == Neumann
        A[1, 2] = -2 * λ
        A[end, end - 1] = -2 * λ
    end 
    
    return A
        
end

build_tri_diagonal_matrix (generic function with 1 method)

In [5]:
function solve_rxn_diffn_eqn(f, u₀, bc::BoundaryCondition, D, Nₓ, st::SpaceTime)
    
    discretization = Discretization(NaN, Nₓ, NaN, NaN)
    
    @assert(bc.ū₀ ≈ u₀(bc.ū₀), "The boundary condition is inconsistent with the initial condition.")
    @assert(bc.ūₗ ≈ u₀(bc.ūₗ), "The boundary condition is inconsistent with the initial condition.")
    x = collect(linspace(0, st.L, discretization.Nₓ))
    discretization.Δx = x[2] - x[1]
    discretization.Δt = discretization.Δx^2
    @printf("%d points in x-discretization. dx = %f\n", discretization.Nₓ, discretization.Δx)
    discretization.Nₜ = ceil(Int, st.tf / discretization.Δt) #this includes the t = 0 step, take Nₜ time steps
    @printf("%d points in t-discretization. dt = %f\n", discretization.Nₜ, discretization.Δt)
    t = [discretization.Δt * (k - 1) for k = 1:discretization.Nₜ]; #list comprehension 
    
    λ = (D * discretization.Δt) / (2 * (discretization.Δx)^2)
    
    ## Making Tridiagonal Matrix ##
    A = build_tri_diagonal_matrix(discretization, λ, bc)
    
    ## Making u ##
    u = zeros(Float64, discretization.Nₓ, discretization.Nₜ) 
    #Take into account the initial condition and boundary counditions
    u[:,1] = u₀.(x) 
    u[1,:] = bc.ū₀
    u[end,:] = bc.ūₗ
       
    ## Making right hand side to fill in the rest of u ##
    if typeof(bc) == Dirichlet
        rhs = zeros(Float64, discretization.Nₓ - 2)
        for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 2:(discretization.Nₓ - 1) #
                rhs[i-1] = λ*(u[i - 1, k - 1] - 2*u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1] 
            end
        u[2:end-1, k] = A \ rhs       #Au = rhs
        end
    end
    
    if typeof(bc) == Periodic 
        rhs = zeros(Float64, discretization.Nₓ - 1)
         for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 1:(discretization.Nₓ - 1) 
                if (i == 1)
                    rhs[i] = λ*(u[Nₓ - 1, k - 1] - 2*u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1] 
                else
                    rhs[i] = λ*(u[i - 1, k - 1] - 2*u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1]
                end
            end
        u[2:end-1, k] = A \ rhs       #Au = rhs
        end
    end
     
    
    if typeof(bc) == Neumann 
        rhs = zeros(Float64, discretization.Nₓ)
        for k = 2:discretization.Nₜ #this is the column of the u array, not the tₖ  
            for i = 1:(discretization.Nₓ) 
                if (i == 1)
                    rhs[i] = λ*((-4 * discretization.Δx * du₀) + 2*u[i + 1, k - 1] - 2*u[i, k - 1]) + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1]
                elseif (i == discretization.Nₓ)
                    rhs[i] = λ*((4 * discretization.Δx * duₗ) + 2*u[i - 1, k - 1] - 2*u[i, k - 1])  + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1]
                else
                    rhs[i] = λ*(u[i - 1, k - 1] - 2*u[i, k - 1] + u[i + 1, k - 1]) + f(x[i], t[k - 1], u[i, k - 1])*discretization.Δt + u[i, k - 1]
                end
            end
        u[2:end-1, k] = A \ rhs       #Au = rhs
        end

    end 
    
        
    return t, x, u 
    
end 

solve_rxn_diffn_eqn (generic function with 1 method)

Enter the following: 

Reaction Term (f(x, t, u))

Initial Condition (u₀)

Boundary Conditions (ū₀ and ūₗ)

Diffusion Coefficient (D)

Number of Spacial Steps (Nₓ)

Length of Simulation (tf) - should be space time?

In [10]:
#Neumann BC Test

g(x::Float64) = x^3

function exact_u(x::Float64, t::Float64) 
    return e^(-π^2 * t) * cos(π * x) + g(x)
end

f(x::Float64, t::Float64, u::Float64) = -g(x)
u₀(x::Float64) = cos(π * x) * e^(-π^2 * t) + g(x)

bc = Neumann(0.0, 3.0)
D = 1.0
Nₓ = 20.0
st = SpaceTime(1.0, 1.0)


t, x, u = solve_rxn_diffn_eqn(f, u₀, bc, D, Nₓ, st)

LoadError: [91mUndefVarError: t not defined[39m

In [6]:
#Dirichlet BC Test
f(x::Float64, t::Float64, u::Float64) = 100 * (D * π^2 - 1.0) * (e ^ (-t)  * sin(π * x))
u₀(x::Float64) = 100 * sin(π * x)

D = 1.0
const Nₓ = 20


20

In [15]:
#d = Discretization(0, 20, 0.3, 0.09)
#bc = Neumann(2.0, 2.0)

Neumann(2.0, 2.0)

In [10]:
bc = Dirichlet(0.0,0.0)

Dirichlet(0.0, 0.0)

In [11]:
st = SpaceTime(1.0, 1.0)

SpaceTime(1.0, 1.0)

In [12]:
t, x, u = solve_rxn_diffn_eqn(f, u₀, bc, D, Nₓ, st)

LoadError: [91mInexactError()[39m

In [10]:
function exact_u(x::Float64, t::Float64) 
        return 100 * exp(-t) * sin(π * x) #exp(-2 * t) * sin(5 * π * x) #Let L = 1
    end

exact_u (generic function with 1 method)

In [14]:
 ## Making gif to compare exact equation and rxn_diffn_equ ##

Nₜ=ceil(Int, tf / Δt)

@gif for i = 1:200:Nₜ
    scatter(x,u[:,i], ylim=(0,100), label="numerical", xlabel="x", ylabel="u", title=@sprintf("t = %.2f", t[i]))
    plot!(x, exact_u.(x,t[i]), label="exact")
end 

LoadError: [91mUndefVarError: tf not defined[39m

In [15]:
import PyPlot; const plt = PyPlot

PyPlot

In [16]:
## Create a heat map ##
Nₜ = length(t)


X = similar(transpose(u))
T = similar(transpose(u))

for i = 1:Nₜ
    X[i,:] = x
end

for i = 1:Nₓ
    T[:,i] = t
end

plt.figure()
plt.pcolormesh(X,T, transpose(u))
plt.xlabel("x")
plt.ylabel("t")
plt.colorbar(label="Concentration of A")

LoadError: [91mUndefVarError: t not defined[39m